In [2]:
# ✅ Cell 1: Install dependencies (run once)
%pip install transformers datasets peft accelerate bitsandbytes -q


Note: you may need to restart the kernel to use updated packages.


In [4]:
# ✅ Cell 2: Imports
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq
from peft import get_peft_model, LoraConfig, TaskType
import torch


In [6]:
# ✅ Cell 3: Load feedback-based fine-tune data
dataset = load_dataset("json", data_files="finetune_data.jsonl", split="train")
dataset[0]


Generating train split: 10 examples [00:00, 1104.11 examples/s]


{'prompt': 'Who are the high-risk customers mentioned in the latest risk assessments?',
 'completion': 'The high-risk customers mentioned in the latest risk assessments are:\n\n1. BioMed Research (Customer ID: 3): This account generates $12M in revenue, but is currently under regulatory scrutiny in the biotech sector.\n2. NextGen Pharmaceuticals (Customer ID: 6): This account brings in $45M in revenue, but has pending FDA approvals.\n3. Digital Health Corp (Customer ID: 9): This account contributes $18M in revenue, but there are data privacy concerns associated with it.\n\nThe total exposure for these high-risk accounts is $75M in annual revenue. The recommended actions include increasing monitoring frequency for these accounts, reviewing contract terms for regulatory compliance clauses, and scheduling quarterly business reviews (Source 1: risk_assessment.eml).'}

In [7]:
# ✅ Cell 4: Load model and tokenizer
model_id = "google/flan-t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_id)
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_id)


In [11]:
# ✅ Cell 5: Configure LoRA + PEFT
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)

model = get_peft_model(base_model, peft_config)
model.print_trainable_parameters()


trainable params: 344,064 || all params: 77,305,216 || trainable%: 0.4451


In [12]:
# ✅ Cell 6: Preprocess data
def preprocess(example):
    inputs = tokenizer(example["prompt"], padding="max_length", truncation=True, max_length=256)
    targets = tokenizer(example["completion"], padding="max_length", truncation=True, max_length=256)
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_dataset = dataset.map(preprocess, remove_columns=["prompt", "completion"])


Map: 100%|██████████| 10/10 [00:00<00:00, 381.62 examples/s]


In [13]:
# ✅ Cell 7: Training arguments
training_args = TrainingArguments(
    output_dir="./models/lora_adapter",
    per_device_train_batch_size=2,
    num_train_epochs=5,
    learning_rate=1e-4,
    logging_dir="./logs",
    save_total_limit=1,
    save_strategy="epoch"
)


In [14]:
# ✅ Cell 8: Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model)
)

trainer.train()
model.save_pretrained("./models/lora_adapter")

print("✅ Fine-tuned model saved to: models/lora_adapter/")


No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/Users/moulika/allyin-compass/venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss


/Users/moulika/allyin-compass/venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/moulika/allyin-compass/venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/moulika/allyin-compass/venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/moulika/allyin-compass/venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


✅ Fine-tuned model saved to: models/lora_adapter/
